In [ ]:
import keras
import os
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
imbd_dir='C:\programowanie\python\deep learning\\6\data\\aclImdb'
train_dir=os.path.join(imbd_dir,'train')

label=[]
texts=[]

for label_type in ['neg','pos']:
    dir_name=os.path.join(train_dir,label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:]=='.txt':
            f=open(os.path.join(dir_name,fname))
            texts.append(f.read())
            f.close()
            if label_type=='neg':
                label.append(0)
            else:
                label.append(1)

In [ ]:
maxlen=100
training_samples=200
validation_sample=10000
max_words=10000

tokenizer=Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences=tokenizer.texts_to_sequences(texts)

word_index=tokenizer.word_index
print(f'Znaleziono {len(word_index)} unikatowych tokenów')

data=pad_sequences(sequences,maxlen=maxlen)

label=np.array(label)
print(data.shape)
print(label.shape)

indices=np.arange(data.shape[0])
np.random.shuffle(indices)
data=data[indices]
label=label[indices]

xtrain=data[:training_samples]
ytrain=label[:training_samples]
xval=data[training_samples:training_samples+validation_sample]
yval=label[training_samples:training_samples+validation_sample]

In [ ]:
glove_dir='C:\programowanie\python\deep learning\\6\data'
#nlp.stanford.edu/projects/glove
embedding_index=[]
f=open(os.path.jpon(glove_dir,'glove.6B.100d.txt'))

for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embedding_index[word]=coefs
f.close()
print(f'Znaleziono {len(embedding_index)} unikatowych tokenów')

In [ ]:
embedding_dim= 100
embedding_matrix=np.zeros((max_words,embedding_dim))

for word, i in word_index.items():
    embedding_vector=embedding_index.get(word)
    if i< max_words:
        if embedding_vector is not None:
            embedding_index[i]=embedding_vector
            

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Flatten,Embedding

In [ ]:
model=Sequential()
model.add(Embedding(max_words,embedding_dim,input_length=maxlen))
model.add(Flatten())
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.summary()

In [ ]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable=False
model.compile(optimizer='rmsprop',
            loss='binary_crossentropy',
            metrics=['acc'])
history=model.fit(xtrain,ytrain,
                  epochs=10,
                  batch_size=32,
                  validation_data=(xval,yval)  )
model.save_weights('pre_trainded_glove_model.h5')